### Example of multioutput model using the functional API - Housing dataset

Loading the dataset and standardizing data 

In [6]:
# Boston Housing dataset - load data
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

# standardize the data
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

(404, 13)
(102, 13)
50.0 5.0


Create, train and estimate the error of a model for the Housing dataset using the functional API

In [8]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

# create model using functional API
input_tensor = Input(shape=(train_data.shape[1],))
x = layers.Dense(64, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
output_tensor = layers.Dense(1,activation= "linear")(x)

model = Model(input_tensor, output_tensor)
model.summary()

# train and test - same as before ...
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

model.fit(train_data, train_targets, epochs=20, batch_size=16, verbose=1)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 13)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,121 (20.00 KB)

 Trainable params: 5,121 (20.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - loss: 495.8739 - mae: 20.4413
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 326.0435 - mae: 16.0553
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - loss: 164.5565 - mae: 10.4502
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - loss: 73.5702 - mae: 6.6281
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step - loss: 43.1088 - mae: 4.7348
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - loss: 35.7023 - mae: 4.2007
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - loss: 24.9264 - mae: 3.5713
Epoch 8/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - loss: 23.3437 - mae: 3.3921
Epoch 9/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: 15.9926 - mae: 2.9113
Epoch 10/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - loss: 15.8528 - mae: 2.8257
Epoch 11/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18.9091 - mae: 2.9063  
Epoch 12/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step - loss: 12.8843 - mae: 2.5430
Epoch 1

Now, let us create a new binary variable to make a multi-output problem

In [10]:
import numpy as np
train_targets_bin = np.digitize(train_targets,bins=[20])
test_targets_bin = np.digitize(test_targets,bins=[20])

In [11]:
# model with two different outputs: regression and binary classification
input_tensor = Input(shape=(train_data.shape[1],))
x = layers.Dense(64, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
output_tensor1 = layers.Dense(1,activation= "linear", name= "output1")(x)
output_tensor2 = layers.Dense(1,activation= "sigmoid", name = "output2")(x)

model = Model(input_tensor, [output_tensor1, output_tensor2])
model.summary()

model.compile(optimizer='rmsprop', loss=['mse','binary_crossentropy'], loss_weights = [0.05, 1])

model.fit(train_data, [train_targets, train_targets_bin], epochs=20, batch_size=16)

print( model.evaluate(test_data, [test_targets, test_targets_bin]) )


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │        896 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      4,160 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output1 (Dense)     │ (None, 1)         │         65 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output2 (Dense)     │ (None, 1)         │         65 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,186 (20.26 KB)

 Trainable params: 5,186 (20.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 26.2821 - output1_loss: 25.6529 - output2_loss: 0.5928
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - loss: 19.0295 - output1_loss: 18.5597 - output2_loss: 0.4858
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - loss: 11.2020 - output1_loss: 10.8031 - output2_loss: 0.3908
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - loss: 5.8211 - output1_loss: 5.4229 - output2_loss: 0.3904
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - loss: 3.3150 - output1_loss: 2.9518 - output2_loss: 0.3588
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 2.2768 - output1_loss: 1.9167 - output2_loss: 0.3572
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - loss: 1.8209 - output1_loss: 1.5280 - output2_loss: 0.2916
Epoch 8/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - loss: 1.6409 - output1_loss: 1.2861 - output2_loss: 0.3624
Epoch 9/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - loss: 1.2605 - output1_loss: 0.9281 